# 투표기반 앙상블

투표기반 앙상블은 여러 분류기를 학습 시킨후 (분류/회귀 둘다 가능) 후 각각의 예측 범주에서 가장 많이 나오는 범주를 예측하는 방법.

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 20)
pd.set_option("display.max_rows", 20)
pd.set_option("display.width", 2000)

# 라이브러리 및 데이터 불러오기
df = pd.read_csv('./extrafiles/Train.csv', engine='python')
df

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
0,1,D,Flight,4,2,177,3,low,F,44,1233,1
1,2,F,Flight,4,5,216,2,low,M,59,3088,1
2,3,A,Flight,2,2,183,4,low,M,48,3374,1
3,4,B,Flight,3,3,176,4,medium,M,10,1177,1
4,5,C,Flight,2,2,184,3,medium,F,46,2484,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10994,10995,A,Ship,4,1,252,5,medium,F,1,1538,1
10995,10996,B,Ship,4,1,232,5,medium,F,6,1247,0
10996,10997,C,Ship,5,4,242,5,low,F,4,1155,0
10997,10998,F,Ship,5,2,223,6,medium,M,2,1210,0


In [4]:
# 시험용 데이터 분리
X_train = df.loc[:8000, 'Warehouse_block':'Weight_in_gms']
y_train = df.loc[:8000, 'Reached.on.Time_Y.N']
X_test = df.loc[8000:, 'Warehouse_block':'Weight_in_gms']
y_test = df.loc[8000:, 'Reached.on.Time_Y.N']
X_test_id = df.loc[8000:, 'ID']
# 이상치 교정
# X_train 의 Discount_offered 컬럼의 이상치 확인

# 사분위 편차를 확인 하여 조정
X_25 = X_train['Discount_offered'].quantile(0.25)
X_75 = X_train['Discount_offered'].quantile(0.75)
X_diff = (X_75 - X_25) * 1.5

# 이상치의 제거
# X_train = X_train[(X_train['Discount_offered'] < X_75 + X_diff) &
#                   (X_train['Discount_offered'] > X_25 - X_diff) ]
# X_train

# 이상치 대체
X_train['Discount_offered'][(X_train['Discount_offered'] > X_75 + X_diff)] = X_75 + X_diff
X_train['Discount_offered'][(X_train['Discount_offered'] < X_25 - X_diff)] = X_25 - X_diff
X_train

,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,D,Flight,4,2,177,3,low,F,44.0,1233
1,F,Flight,4,5,216,2,low,M,51.5,3088
2,A,Flight,2,2,183,4,low,M,48.0,3374
3,B,Flight,3,3,176,4,medium,M,10.0,1177
4,C,Flight,2,2,184,3,medium,F,46.0,2484
...,...,...,...,...,...,...,...,...,...,...
7996,C,Ship,5,3,262,5,low,F,8.0,1493
7997,F,Ship,6,1,306,4,low,F,4.0,1449
7998,D,Ship,6,4,276,4,low,F,2.0,1120
7999,F,Ship,3,2,183,5,low,F,2.0,1158


In [6]:
# 더미변수화
X_train.columns
# 'ID', 'Warehouse_block', 'Mode_of_Shipment', 'Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product', 'Prior_purchases', 'Product_importance', 'Gender', 'Discount_offered', 'Weight_in_gms

# train 데이터 변환
X_num_train = X_train[['Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product', 
           'Prior_purchases', 'Discount_offered', 'Weight_in_gms']]
X_cat_train = X_train[['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender']]
X_cat_train = pd.get_dummies(X_cat_train)
X_cat_train

# test 데이터 변환
X_num_test = X_test[['Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product', 
           'Prior_purchases', 'Discount_offered', 'Weight_in_gms']]
X_cat_test = X_test[['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender']]
X_cat_test = pd.get_dummies(X_cat_test)
X_cat_test

,Warehouse_block_A,Warehouse_block_B,Warehouse_block_C,Warehouse_block_D,Warehouse_block_F,Mode_of_Shipment_Flight,Mode_of_Shipment_Road,Mode_of_Shipment_Ship,Product_importance_high,Product_importance_low,Product_importance_medium,Gender_F,Gender_M
8000,1,0,0,0,0,0,0,1,0,1,0,0,1
8001,0,1,0,0,0,0,0,1,0,1,0,0,1
8002,0,0,1,0,0,0,0,1,0,1,0,1,0
8003,0,0,0,0,1,0,0,1,0,0,1,0,1
8004,0,0,0,1,0,0,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10994,1,0,0,0,0,0,0,1,0,0,1,1,0
10995,0,1,0,0,0,0,0,1,0,0,1,1,0
10996,0,0,1,0,0,0,0,1,0,1,0,1,0
10997,0,0,0,0,1,0,0,1,0,0,1,0,1


In [7]:
# 스케일링
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(X_num_train)

# train 데이터 변환
X_train_scaled = scaler.transform(X_num_train)
X_train_scaled

# test 데이터 변환
X_test_scaled = scaler.transform(X_num_test)
X_test_scaled

# 데이터 병합
X_train = pd.concat([X_num_train, X_cat_train], axis=1)
X_test = pd.concat([X_num_test, X_cat_test], axis=1)

# 데이터 준비 완성
X_train

,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms,Warehouse_block_A,Warehouse_block_B,Warehouse_block_C,Warehouse_block_D,Warehouse_block_F,Mode_of_Shipment_Flight,Mode_of_Shipment_Road,Mode_of_Shipment_Ship,Product_importance_high,Product_importance_low,Product_importance_medium,Gender_F,Gender_M
0,4,2,177,3,44.0,1233,0,0,0,1,0,1,0,0,0,1,0,1,0
1,4,5,216,2,51.5,3088,0,0,0,0,1,1,0,0,0,1,0,0,1
2,2,2,183,4,48.0,3374,1,0,0,0,0,1,0,0,0,1,0,0,1
3,3,3,176,4,10.0,1177,0,1,0,0,0,1,0,0,0,0,1,0,1
4,2,2,184,3,46.0,2484,0,0,1,0,0,1,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7996,5,3,262,5,8.0,1493,0,0,1,0,0,0,0,1,0,1,0,1,0
7997,6,1,306,4,4.0,1449,0,0,0,0,1,0,0,1,0,1,0,1,0
7998,6,4,276,4,2.0,1120,0,0,0,1,0,0,0,1,0,1,0,1,0
7999,3,2,183,5,2.0,1158,0,0,0,0,1,0,0,1,0,1,0,1,0


In [9]:
# 모델 적용
# 로지스틱 회귀, 랜덤 포레스트, 서포트 백터 3가지를 믹싱 . 베이지안도 고려 해보기
# 선형판별, 선형판별, 판별함수


from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

model_logi = LogisticRegression()
model_svc = SVC()
model_forest = RandomForestClassifier()

# hard voting 은 범주 기반 분류, soft voting 은 확률 기반 분류
model_voting = VotingClassifier(estimators=[('a', model_logi), ('b', model_svc), ('c', model_forest)], voting='hard')
model_voting.fit(X_train, y_train)

VotingClassifier(estimators=[('a', LogisticRegression()), ('b', SVC()),
                             ('c', RandomForestClassifier())])

In [11]:
# model.score()
print(model_voting.score(X_train, y_train))

0.8083989501312336


In [15]:
# 교차검증
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5, random_state=333, shuffle=True)
score = cross_val_score(VotingClassifier(estimators=[('a', model_logi), ('b', model_svc), ('c', model_forest)], voting='hard'), X_train, y_train, cv=kfold)

print(score)

[0.70955653 0.71625    0.69875    0.705625   0.724375  ]


In [16]:
print(score.mean())

0.7109113054341037
